<a href="https://colab.research.google.com/github/fivearm/Phase_4_Project/blob/ninas_branch/check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose
#from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%matplotlib inline

Mounted at /content/gdrive


In [177]:
home_value_old = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/zillow_data.csv')
home_value_new = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

mean_listing_price_cut = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_mean_listings_price_cut_amt_uc_sfrcondo_sm_month.csv')
median_listing_price_cut = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_med_listings_price_cut_amt_uc_sfrcondo_sm_month.csv')
perc_listing_with_price_cut = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_perc_listings_price_cut_uc_sfrcondo_sm_month.csv')

for_sale_inv_smoothed = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_invt_fs_uc_sfrcondo_sm_month.csv')

median_sale_price = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/Metro_median_sale_price_uc_sfrcondo_sm_sa_month.csv')
median_listing_price= pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/median_list_price.csv')

newly_pending_listings = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/newly_pending_listings.csv')
mean_days_to_pending = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/mean_days_to_pending.csv')
median_days_to_pending = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/median_days_to_pending.csv')

lumber = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/plywood_prices.csv').rename({'WPU083': 'plywood prices', 'DATE': 'time'}, axis=1).set_index('time')
concrete = pd.read_csv('/content/gdrive/MyDrive/Phase_4_Project/data/concrete_prices.csv').rename({'PCU32733273': 'concrete prices', 'DATE' : 'time'}, axis = 1).set_index('time')


new_dfs = [for_sale_inv_smoothed
          ,mean_listing_price_cut
          ,median_listing_price_cut
          ,median_sale_price
          ,perc_listing_with_price_cut
          ,newly_pending_listings
          ,mean_days_to_pending
          ,median_days_to_pending
          ,median_listing_price
          ,home_value_old
          ,home_value_new]
          

non_date_cols = [col 
                  for df in new_dfs 
                 for col in df.columns
                 if not col[:3].isdigit()]
id_cols = list(set(non_date_cols))






In [178]:
def melt_data(df, compress = False, _id_vars=None):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    
    If compress is True will compress data to only the value column.
    """
    if not _id_vars:
      _id_vars = ['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 
                                        'Metro', 'CountyName']
    present_vars = df.columns[df.columns.isin(_id_vars)]
    if not present_vars.empty:

      melted = pd.melt(df,id_vars=present_vars, var_name='time')
    else:
      melted = pd.melt(df, var_name='time')
    
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True).apply(lambda x: x.strftime('%Y-%m'))
    melted = melted.dropna(subset=['value'])    
    resampled = melted.groupby(['RegionID','time'])['value'].mean()

    return resampled

In [ ]:
melted_new = [melt_data(df, _id_vars=id_cols) for df in new_dfs]

names = ['for_sale_inv'
          ,'mean_listing_price_cut'
          ,'median_listing_price_cut'
          ,'median_sale_price'
          ,'perc_listing_with_price_cut'
          ,'newly_pending_listings'
          ,'mean_days_to_pending'
          ,'median_days_to_pending'
          ,'median_listing_price'
          ,'home_value'
          ,'home_value'
          ]
combined = pd.concat(melted_new,axis='columns', join='outer', keys=names).dropna(how='all')
combined

In [183]:
combined.iloc[:,-2] = combined.iloc[:,-2].fillna(combined.iloc[:,-1])
combined_filtered = combined.iloc[:,:-1]

In [198]:
con_lum = pd.concat([concrete, lumber], axis=1)
con_lum.index = pd.to_datetime(con_lum.index)
con_lum_resamp = con_lum.resample('M').mean().reset_index()
con_lum_resamp['time'] = con_lum_resamp['time'].apply(lambda x: x.strftime('%Y-%m'))

final_df = combined_filtered.reset_index().merge(con_lum_resamp, right_on='time', left_on='time', how='left').set_index(['RegionID', 'time']).dropna(thresh=8)

final_df.corr()['home_value']

for_sale_inv                  -0.008658
mean_listing_price_cut         0.851738
median_listing_price_cut       0.869375
median_sale_price              0.985147
perc_listing_with_price_cut   -0.070317
newly_pending_listings         0.143695
mean_days_to_pending          -0.259710
median_days_to_pending        -0.154947
median_listing_price           0.945003
home_value                     1.000000
concrete prices                0.022859
plywood prices                 0.022047
Name: home_value, dtype: float64

In [ ]:
def kpss_test(df, name = None):
    '''
    Will return the kpss test
    
    Parmeters
    ----------
    df = dataframe or series
    name = Name of the variable you are testing
    '''
    df_test = kpss(np.log(df), nlags='auto')
    print(f'Is {name} data stationary?')
    print(f'Test stastic = {df_test[0]: .3f}')
    print(f'P-value = {df_test[1]: .3f}')
    print('Critical Values: ')
    for k, v in df_test[3].items():
        print(f'{k}: {v}')

## Stationarize and Outlier Handling


In [206]:
def is_stationary(df):
  df_test = adfuller(df)
  critical_95 = df_test[4]['5%']
  return critical_95 > df_test[0] 

def make_stationary(df):
  log_transf = np.log2(df)
  
  diff = log_transf.diff().dropna()
  while is_stationary(diff) == False:
    diff = diff.diff().dropna()
  diff.name = df.name+ '_stationary'
  return diff

### Looks like plywood and cement prices are non stationary

In [201]:
for col in final_df.columns:
  if is_stationary(final_df[col].dropna()) == False:
    print(f'{col} is non stationary')

concrete prices is non stationary
plywood prices is non stationary


In [214]:
 stationary_concrete = make_stationary(final_df['concrete prices'])
 stationary_plywood = make_stationary(final_df['plywood prices'])
 
final_df = final_df.merge(stationary_concrete,left_index=True, right_index=True)
final_df = final_df.merge(stationary_plywood,left_index=True, right_index=True).drop(columns=['concrete prices','plywood prices'])

In [210]:
def replace_with_median(col, lower_threshold, median):
    col[col > -lower_threshold] =  median
    col[col < lower_threshold] = median
    return col

def get_replacement_vals(col):
      std = col.std(ddof=0)
      mean = col.mean()
      median = col.median()
      lower_thresh = (-3 * std)+ mean
      return median, lower_thresh

def replace_outliers(train, test=pd.DataFrame()):
  train_copy = train.copy()
  median, lower_thresh = get_replacement_vals(train_copy)
  stand_train = replace_with_median(train_copy, lower_thresh, median)

  if not test.empty:
    test_copy = test.copy()
    stand_test = replace_with_median(test_copy, lower_thresh, median)
    return stand_train, stand_test
  
  return stand_train


KeyError: ignored

## Cross Validation

In [ ]:
def rmse(y_true, y_pred):
    sum_squared_diff = sum([(y2 - y1)**2 for y2,y1 in zip(y_true, y_pred)])
    rmse = (sum_squared_diff * (1/len(y_true))) ** 0.5
    return rmse

def cross_validate(model, df, split_freq='M', model_kwargs={}):
    '''
    Resamples dataframe using aggregate mean
    Preforms cross validation of the input model. 
    ---------------------------------------------------------------------
    model: uninitalized model- no parantheses 
    split_freq: the frequency to index the data by
    model_kwargs: keyword arguments to pass into the model before fitting
    ---------------------------------------------------------------------
    Returns: An array of RMSE
    '''
    
    df = df.resample(split_freq).mean()
    all_rmse = []
    # Create and loop through 4 different train and validation sets
    split = TimeSeriesSplit(n_splits=4)
    for train_ind, val_ind in split.split(df):
        # Initialize, fit, and generate predictions the model on subset of df
        model_fit = model(df.iloc[train_ind], **model_kwargs).fit()
        preds = model_fit.predict(typ='levels', start=val_ind[0], end=val_ind[-1])
        
        # Grab Y actual from df
        true = df.iloc[val_ind]
        
        # Calculate and store RMSE
        all_rmse.append(rmse(true, preds))
    return all_rmse 

cross_validate(ARIMA, Miami_df['value'], 'M', model_kwargs={'order':(1,2,0)})

## Arima 

In [ ]:
def arima(series, order):
    '''
    order = (p,d,q)
    p: The number of lag observations included in the model, also called the lag order.
    d: The number of times that the raw observations are differenced, also called the degree of differencing.
    q: The size of the moving average window, also called the order of moving average
    '''
    model = ARIMA(series.values, order=order).fit()
    print(model.summary())
    
    residuals = model.resid
    fig, ax = plt.subplots()
    ax.plot(series.index, residuals)